In [7]:
import numpy as np

import import_ipynb
from grid_world import windy_grid, ACTION_SPACE
from iterative_policy_evaluation_probabilistic import print_values, print_policy

importing Jupyter notebook from iterative_policy_evaluation_probabilistic.ipynb


In [2]:
SMALL_ENOGH = 1e-3
GAMMA = 0.9

In [15]:
def get_transition_probs_and_rewards(grid) :
    transition_probs = {}
    rewards = {}
    for (s, a), v in grid.probs.items() :
        for s2, p in v.items() :
            transition_probs[(s, a, s2)] = p
            rewards[(s, a, s2)] = grid.rewards.get(s2, 0)
            
    return transition_probs, rewards

In [16]:
if __name__ == '__main__' :
    grid = windy_grid()
    transition_probs, rewards = get_transition_probs_and_rewards(grid)
    
    V = {}
    for s in grid.all_states() :
        V[s] = 0
        
    while True :
        biggest_change = 0
        
        for s in grid.actions.keys() :
            old_v = V[s]
            new_v = float('-inf')

            for a in ACTION_SPACE :
                v = 0
                for s2 in grid.all_states() :
                    r = rewards.get((s, a, s2), 0)
                    v += transition_probs.get((s, a, s2), 0) * (r + GAMMA * V[s2])
                if v > new_v :
                    new_v = v
                    
            V[s] = new_v
            biggest_change = max(biggest_change, np.abs(old_v - V[s]))
            
        if biggest_change < SMALL_ENOGH :
            break
    
    #find the optimal policy
    policy = {}
    for s in grid.actions.keys() :
        best_a = None
        best_value = float('-inf')
        
        for a in ACTION_SPACE :
            v = 0
            for s2 in grid.all_states() :
                r = rewards.get((s, a, s2), 0)
                v += transition_probs.get((s, a, s2), 0) * (r + GAMMA * V[s2])
            if v > best_value :
                best_value = v
                best_a = a
        policy[s] = best_a
        
    print('Values')
    print_values(V, grid)
    print('Policy')
    print_policy(policy, grid)

Values
+-------+-------+-------+-------+
| 0.810 | 0.900 | 1.000 | 0.000 |
+-------+-------+-------+-------+
| 0.729 | 0.000 | 0.478 | 0.000 |
+-------+-------+-------+-------+
| 0.656 | 0.590 | 0.531 | 0.478 |
+-------+-------+-------+-------+
Policy
+---+---+---+---+
| R | R | R |   |
+---+---+---+---+
| U |   | D |   |
+---+---+---+---+
| U | L | L | L |
+---+---+---+---+
